In [24]:
import numpy as np
from types import SimpleNamespace

# Fixed parameters
phi = 0
phik = 0.8
r = 2.5 / 400
sig = 1 / 400
gam = 4
del_ = 0.0129
alp = 0.7
zet = 0.15

# Numerical parameters
eps = 1e-3
tol = 1e-12
min_it_V = 100
max_it_V = 300
tol_V = 1e-4
tol_prob = 1e-7
It_par = 100
tol_par = 1e-4

# Calibration targets
AGDP_target = 1.78
Frisch_target = 1
En_target = 0.40
un_target = 0.40
Targets = np.array([0, AGDP_target, En_target, Frisch_target, un_target])

gr = SimpleNamespace()
# Load theta process (inc_process)
pr = np.array([0.099262392990694, 0.239835302385175, 0.321804609248276, 0.239835302385175, 0.099262392990694])

# Define the list Pr
Pr = [
    [0.926078033773890, 0.0739215065331679, 4.59692942378354e-07, 0, 0],
    [0.0305944855213048, 0.913356724647118, 0.0560485686929237, 2.21138653166975e-07, 0],
    [1.04334586346957e-07, 0.0417720476612774, 0.916455696008273, 0.0417720476612775, 1.04334586303878e-07],
    [8.63978491220093e-18, 2.21138653200695e-07, 0.0560485686929237, 0.913356724647118, 0.0305944855213047],
    [1.36444779322266e-32, 2.90263425432067e-17, 4.59692942301714e-07, 0.0739215065331678, 0.926078033773890]
]

# Convert the list to a 5x5 NumPy array
Pr = np.array(Pr)

x = np.array([-0.918119742420827, -0.459059871210413, 0, 0.459059871210413, 0.918119742420827])

the = np.concatenate(([0], np.exp(x)))
S = len(the)

fin = 0.8820
sep = 0.0573

Pr = np.zeros((S, S))
Pr[0, 0] = 1 - fin
Pr[0, 1:] = fin * pr
Pr[1:, 0] = sep
Pr[1:, 1:] = (1 - sep) * np.eye(S-1)
pr = np.ones(S) / S
dis = 1

# Compute invariant distribution for theta
while dis > tol:
    pri = pr @ Pr
    dis = np.max(np.abs(pri - pr))
    pr = pri

print(pr)

# Grids
I_k = 150
kl = 1
kh = 10
I_b = 200
bl = -10
bh = 30
I_fill = 50

gr_b = np.linspace(bl, bh, I_b)
db = gr_b[1] - gr_b[0]
gr_k = np.linspace(kl, kh, I_k)
dk = gr_k[1] - gr_k[0]

bin_zero = 1 + (0 - bl) / db
bind_zero = max(int(bin_zero), 1)
bw_zero = bin_zero - bind_zero

# Save grid parameters
grids = {
    'gr_b': gr_b,
    'gr_k': gr_k,
    'I_b': I_b,
    'bl': bl,
    'bh': bh,
    'db': db,
    'I_k': I_k,
    'kl': kl,
    'kh': kh,
    'dk': dk,
    'I_fill': I_fill,
    'bin_zero': bin_zero,
    'bind_zero': bind_zero,
    'bw_zero': bw_zero
}

bin_constr = 1 + (-phi - phik * gr_k - bl) / db
bmesh, kmesh = np.meshgrid(gr_b, gr_k)
check = (bmesh + phi + phik * kmesh >= 0)
n_check = S * np.sum(check)
check = np.repeat(check[:, :, np.newaxis], S, axis=2)

# # Initial conditions
EVk = np.tile((1.0 / (1 + gr_k))[:, np.newaxis] @ np.ones((1, I_b)), (1, 1, S))
EVb = np.tile(np.ones((I_k, 1)) @ (1.0 / (1 + gr_b - bl))[np.newaxis, :], (1, 1, S))
prob = np.ones((I_k * I_b, S)) / (I_k * I_b * S)

# Preallocate matrices
Vk = np.zeros((I_k, I_b, S))
Vb = np.zeros((I_k, I_b, S))
pol_kb = np.zeros((I_k, I_b, S))
wei_kb = np.zeros((4, I_k, I_b, S))
pol_c = np.zeros((I_k, I_b, S))
pol_inv = np.zeros((I_k, I_b, S))
pol_y = np.zeros((I_k, I_b, S))
pol_n = np.zeros((I_k, I_b, S))

# Initial conditions for parameters
bet = 0.93 ** (1 / 4)

En = 0.4
Frisch = 1
eta = 1 / Frisch * (1 - En) / En
den = alp * (1 / bet - 1 + del_) + (1 - alp) * del_
c_av = alp * (1 / bet - 1 + del_) / den * En
k_av = (1 - alp) / den * En
psi = alp * c_av ** (alp * (1 - gam) - 1) * k_av ** ((1 - alp) * (1 - gam)) * En ** eta
const = (the / psi) ** (-1 / eta)

un_ben = 0.4 * (c_av + del_ * k_av)
B_sup = (1.7 - 0.65) * 4 * (c_av + del_ * k_av)

# Iterate on parameters
for it_par in range(It_par):
    q = 1 / (1 + r)
    q_hat = (1 - sig) / (1 + r)
    tax = pr[0] * un_ben + (1 - q) * B_sup
    tau = np.array([tax - un_ben] + [tax] * (S - 1))

    last_round = False
    for it in range(max_it_V):
        for s in range(S):
            # Upper band, lender rate
            bin_l = np.ones(I_k)
            bin_h = (I_b - eps) * np.ones(I_k)
            dif = 1

            while dif > tol:
                bin = (bin_l + bin_h) / 2
                bind = np.floor(bin).astype(int)
                bw = bin - bind
                F1 = (1 - bw) * EVb[np.arange(I_k) + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVb[np.arange(I_k) + I_k * bind + I_k * I_b * (s - 1)]
                F2 = (1 - bw) * EVk[np.arange(I_k) + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVk[np.arange(I_k) + I_k * bind + I_k * I_b * (s - 1)]
                F = F1 - q * F2
                bin_l[F >= 0] = bin[F >= 0]
                bin_h[F < 0] = bin[F < 0]
                dif = np.max(np.abs(bin_h - bin_l))

            bin_UL = np.maximum(bin, bin_zero)

            # upper band, borrower rate
            bin_l = np.ones(I_k)
            bin_h = (I_b - eps) * np.ones(I_k)
            dif = 1
            while dif > tol:
                bin = (bin_l + bin_h) / 2
                bind = np.floor(bin).astype(int)
                bw = bin - bind
                F1 = (1 - bw) * EVb[np.arange(I_k) + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVb[np.arange(I_k) + I_k * bind + I_k * I_b * (s - 1)]
                F2 = (1 - bw) * EVk[np.arange(I_k) + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVk[np.arange(I_k) + I_k * bind + I_k * I_b * (s - 1)]
                F = F1 - q_hat * F2
                bin_l[F >= 0] = bin[F >= 0]
                bin_h[F < 0] = bin[F < 0]
                dif = np.max(np.abs(bin_h - bin_l))

            bin_UB = np.maximum(bin, bin_constr)

            bin_U = np.minimum(bin_UL, bin_UB)

            # lower band, lender rate
            bin_l = np.ones(I_k)
            bin_h = (I_b - eps) * np.ones(I_k)
            dif = 1
            while dif > tol:
                bin = (bin_l + bin_h) / 2
                bind = np.floor(bin).astype(int)
                bw = bin - bind
                F1 = (1 - bw) * EVb[np.arange(I_k) + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVb[np.arange(I_k) + I_k * bind + I_k * I_b * (s - 1)]
                F2 = (1 - bw) * EVk[np.arange(I_k) + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVk[np.arange(I_k) + I_k * bind + I_k * I_b * (s - 1)]
                F = (1 - zet) * F1 - q * F2
                bin_l[F >= 0] = bin[F >= 0]
                bin_h[F < 0] = bin[F < 0]
                dif = np.max(np.abs(bin_h - bin_l))

            bin_LL = np.maximum(bin, bin_zero)

            # lower band, borrower rate
            bin_l = np.ones(I_k)
            bin_h = (I_b - eps) * np.ones(I_k)
            dif = 1
            while dif > tol:
                bin = (bin_l + bin_h) / 2
                bind = np.floor(bin).astype(int)
                bw = bin - bind
                F1 = (1 - bw) * EVb[np.arange(I_k) + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVb[np.arange(I_k) + I_k * bind + I_k * I_b * (s - 1)]
                F2 = (1 - bw) * EVk[np.arange(I_k) + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVk[np.arange(I_k) + I_k * bind + I_k * I_b * (s - 1)]
                F = (1 - zet) * F1 - q_hat * F2
                bin_l[F >= 0] = bin[F >= 0]
                bin_h[F < 0] = bin[F < 0]
                dif = np.max(np.abs(bin_h - bin_l))

            bin_LB = np.maximum(bin, bin_constr)

            bin_L = np.minimum(bin_LL, bin_LB)

            # find current b consistent with each k' being optimal
            # and use envelope conditions to update EVk and EVb

            for i in range(I_k):  # capital today
                k = gr.k[i]
                kind = kind = np.concatenate((np.arange(1, i), np.full(I_fill, i), np.arange(i + 1, I_k)))
                if bin_L[i] == bin_constr[i] and bin_U[i] > bin_constr[i]:
                    bin = np.concatenate((bin_L[:i], [bin_L[i]] * (I_fill // 2), np.linspace(bin_L[i], bin_U[i], I_fill // 2), bin_U[i + 1:]))
                else:
                    bin = np.concatenate((bin_L[:i], np.linspace(bin_L[i], bin_U[i], I_fill), bin_U[i + 1:]))
                bind = np.maximum(np.floor(bin), 1).astype(int)
                bw = bin - bind
                F1 = (1 - bw) * EVb[kind + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVb[kind + I_k * bind + I_k * I_b * (s - 1)]
                F2 = (1 - bw) * EVk[kind + I_k * (bind - 1) + I_k * I_b * (s - 1)] + bw * EVk[kind + I_k * bind + I_k * I_b * (s - 1)]
                ki = gr.k[kind]
                bi = (1 - bw) * gr.b[bind] + bw * gr.b[bind + 1]

                # compute lam and mu
                lam = np.full_like(bin, np.nan)
                mu = np.zeros_like(bin)
                # region k' < k
                j = (bin < bin_zero) & (kind < i)
                mu[j] = bet * (q_hat * F2[j] - (1 - zet) * F1[j]) / (1 - zet - q_hat * phik)
                j = kind < i
                lam[j] = (bet * F2[j] + phik * mu[j]) / (1 - zet)
                # region k' > k
                j = (bin < bin_zero) & (kind > i)
                mu[j] = bet * (q_hat * F2[j] - F1[j]) / (1 - q_hat * phik)
                j = kind > i
                lam[j] = bet * F2[j] + phik * mu[j]

                # region k' = k
                if bin_U[i] == bin_constr[i]:
                    mu_l = bet * (q_hat * F2[i] - (1 - zet) * F1[i]) / (1 - zet - q_hat * phik)
                    mu_u = max(bet * (q_hat * F2[i] - F1[i]) / (1 - q_hat * phik), 0)
                    mu[i:i + I_fill - 1] = np.linspace(mu_l, mu_u, I_fill)
                    if mu_u < 0:
                        print('warning: negative mu')
                elif bin_L[i] == bin_constr[i]:
                    mu_l = bet * (q_hat * F2[i] - (1 - zet) * F1[i]) / (1 - zet - q_hat * phik)
                    mu[i:i + I_fill // 2 - 1] = np.linspace(mu_l, 0, I_fill // 2)

                j = (kind == i) & (bin < bin_zero)
                lam[j] = (bet * F1[j] + mu[j]) / q_hat
                j = (kind == i) & (bin > bin_zero)
                lam[j] = (bet * F1[j] + mu[j]) / q

                if bin_L[i] <= bin_zero and bin_U[i] >= bin_zero:
                    i_z = i + np.where(bin[i:i + I_fill - 1] <= bin_zero)[0][-1]
                    ki = np.concatenate((ki[:i_z], [k] * I_fill, ki[i_z + 1:]))
                    bi = np.concatenate((bi[:i_z], [0] * I_fill, bi[i_z + 1:]))
                    f1 = (1 - bw_zero) * EVb[i + I_k * (bind_zero - 1) + I_k * I_b * (s - 1)] + bw_zero * EVb[i + I_k * bind_zero + I_k * I_b * (s - 1)]
                    f2 = (1 - bw_zero) * EVk[i + I_k * (bind_zero - 1) + I_k * I_b * (s - 1)] + bw_zero * EVk[i + I_k * bind_zero + I_k * I_b * (s - 1)]
                    lam_l = min(bet * f1 / q_hat, bet * f2 / (1 - zet))
                    lam_u = max(bet * f1 / q, bet * f2)
                    lam = np.concatenate((lam[:i_z], np.linspace(lam_l, lam_u, I_fill), lam[i_z + 1:]))
                    F2 = np.concatenate((F2[:i_z], [f2] * I_fill, F2[i_z + 1:]))
                    mu = np.concatenate((mu[:i_z], [0] * I_fill, mu[i_z + 1:]))
                c = (alp * k ** ((1 - alp) * (1 - gam)) / lam) ** (1 / (1 - alp * (1 - gam)))
                b = q * np.maximum(bi, 0) + q_hat * np.minimum(bi, 0) + np.maximum(ki - k, 0) + (1 - zet) * np.minimum(ki - k, 0) + del_ * k + c - the[s] * np.maximum(1 - const[s] * lam ** (-1 / eta), 0) + tau[s]

                # linear interpolation
                b, index = np.sort(b), np.argsort(b)  # re-sort to get non-increasing b
                lam = lam[index]
                mu = mu[index]
                F2 = F2[index]
                c = c[index]
                bi = bi[index]
                ki = ki[index]
                Db = np.diff(b)
                L = len(b)
                ind = np.digitize(gr.b, b) - 1  # assign each gridpoint to a value in b
                ind[gr.b < b[0]] = 0
                ind[gr.b >= b[L - 1]] = L - 2
                wei = (gr.b - b[ind]) / Db[ind]
                wei = np.clip(wei, 0, 1)

                # envelope conditions
                Vb[i, :, s] = (1 - wei) * lam[ind] + wei * lam[ind + 1]
                vk = (1 - alp) * c ** (alp * (1 - gam)) * k ** ((1 - alp) * (1 - gam) - 1) - del_ * lam + bet * F2 + phik * mu
                Vk[i, :, s] = (1 - wei) * vk[ind] + wei * vk[ind + 1]

                # policies
                if last_round == 1 or it == max_it_V:
                    temp = 1 + ((1 - wei) * ki[ind] + wei * ki[ind + 1] - kl) / dk
                    pol_k = np.minimum(np.floor(temp), I_k - 1).astype(int)
                    wei_k = temp - pol_k
                    temp = 1 + ((1 - wei) * bi[ind] + wei * bi[ind + 1] - bl) / db
                    pol_b = np.minimum(np.floor(temp), I_b - 1).astype(int)
                    wei_b = temp - pol_b
                    pol_kb[i, :, s] = pol_k + I_k * (pol_b - 1)
                    wei_kb[0, i, :, s] = (1 - wei_k) * (1 - wei_b)
                    wei_kb[1, i, :, s] = wei_k * (1 - wei_b)
                    wei_kb[2, i, :, s] = (1 - wei_k) * wei_b
                    wei_kb[3, i, :, s] = wei_k * wei_b

                    n = np.maximum(1 - const[s] * lam ** (-1 / eta), 0)
                    inv = np.maximum(ki - k, 0) + (1 - zet) * np.minimum(ki - k, 0) + del_ * k
                    pol_c[i, :, s] = (1 - wei) * c[ind] + wei * c[ind + 1]
                    pol_inv[i, :, s] = (1 - wei) * inv[ind] + wei * inv[ind + 1]
                    pol_n[i, :, s] = (1 - wei) * n[ind] + wei * n[ind + 1]
                    pol_y[i, :, s] = the[s] * pol_n[i, :, s] ** eta * pol_k[i, :, s] ** alp
        
        # update and check convergence
        Temp = np.reshape(Vb, (I_k * I_b, S)) @ Pr.T
        EVbi = np.reshape(Temp, (I_k, I_b, S))
        Temp = np.reshape(Vk, (I_k * I_b, S)) @ Pr.T
        EVki = np.reshape(Temp, (I_k, I_b, S))
        if last_round == 1:
            disEVb = np.abs(EVbi - EVb)
            disEVk = np.abs(EVki - EVk)
            break
        dis = np.abs(EVbi - EVb) + np.abs(EVki - EVk)
        dis = np.sum(dis * check) / n_check
        EVb = EVbi
        EVk = EVki
        if it >= min_it_V and dis < tol_V:
            last_round = 1 

    # Invariant distribution calculation
    crit = 1
    B = 0
    while crit > tol_prob:
        probi = np.zeros((I_k * I_b, S))
        for i in range(I_k):
            for j in range(I_b):
                for s in range(S):
                    ind_kb = int(pol_kb[i, j, s])
                    probi[ind_kb, s] += wei_kb[0, i, j, s] * prob[i * I_b + j, s]
                    probi[ind_kb + 1, s] += wei_kb[1, i, j, s] * prob[i * I_b + j, s]
                    probi[ind_kb + I_k, s] += wei_kb[2, i, j, s] * prob[i * I_b + j, s]
                    probi[ind_kb + 1 + I_k, s] += wei_kb[3, i, j, s] * prob[i * I_b + j, s]
        prob = probi @ Pr

        prob_b = np.sum(np.sum(prob, axis=2), axis=0)
        Bi = np.sum(prob_b * gr_b)
        crit = abs(Bi - B)
        B = Bi

    # Compute statistics and update parameters
    joint = np.sum(prob, axis=2)
    prob_k = np.sum(joint, axis=1)
    prob_b = np.sum(joint, axis=0)
    K = np.sum(prob_k * gr_k)
    B = np.sum(prob_b * gr_b)
    D = -np.sum(prob_b * np.minimum(gr_b, 0))
    A = B + D
    C = np.sum(prob * pol_c)
    Inv = np.sum(prob * pol_inv)
    Y = np.sum(prob * pol_y)
    GDP = 4 * Y
    KGDP = K / GDP
    DGDP = D / GDP
    AGDP = A / GDP
    WGDP = (B + K) / GDP

    # Reshape `pol_n` to a column vector and calculate expected values
    n = pol_n.flatten()
    En = np.sum(prob.flatten() * n * (n > 0)) / np.sum(prob.flatten() * (n > 0))
    n_low = 0.1 * En

    # Calculate the elasticity `ela` and Frisch elasticity
    ela = np.zeros(I_k * I_b * S)
    ela[n > n_low] = 1 / eta * (1 - n[n > n_low]) / n[n > n_low]
    Frisch = np.sum(prob.flatten() * ela) / np.sum(prob.flatten() * (n > n_low))

    # Calculate values of interest `F` and `Targets`
    F = np.array([B - B_sup, AGDP, En, Frisch, un_ben / Y])
    dif_par = np.max(np.abs(F - Targets))

    # Display the results
    print("F:", F)
    print("Targets:", Targets)

    # Check for convergence
    if dif_par < tol_par:
        break

    # Update parameters
    rho = 1 / bet - 1
    rho += 0.2 / 400 * (A - AGDP_target * 4 * Y)
    bet = 1 / (1 + rho)

    B_sup = B
    un_ben = un_target * Y
    eta = eta + 0.5 * (eta * Frisch / Frisch_target - eta)
    psi = psi + 0.5 * (psi * (1 - En_target) ** eta / ((1 - En) ** eta) - psi)

    const = (the / psi) ** (-1 / eta)

[0.06100287 0.0932071  0.22520466 0.3021736  0.22520466 0.0932071 ]


C:\Users\thoma\AppData\Local\Temp\ipykernel_27296\105505088.py:138: RuntimeWarning: divide by zero encountered in power
  const = (the / psi) ** (-1 / eta)


IndexError: index -15150 is out of bounds for axis 0 with size 1